# **Amazon Top 50 Bestselling Books 2009-2019**# 

![Amazon Top 50 Bestselling Books 2009-2019](http://thefutureofink.com/wp-content/uploads/2013/12/AmazonEbooks.png)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Data from the Amazon Top 50 Bestselling Books 2009-2019 dataset.# 

In [ ]:
book = pd.read_csv('../input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv')
book.head()

# Checking the data.# 

In [ ]:
book.info()

**Checking missing values in the data.**

In [ ]:
book.isnull().sum()

**Checking duplicate values in the data.**

In [ ]:
book['Name'].value_counts()

**Removing duplicate values from the data.**

In [ ]:
book.drop_duplicates('Name', keep = 'last', inplace = True)
book.info()

# **Performing EDA on the data.**# 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
book.plot(kind = 'bar', x = 'Year', y = 'Reviews', ax = ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(y='Reviews', x='Year', data=book, ax=ax)

In [ ]:
sns.regplot(x=book['User Rating'], y=book['Price'])

In [ ]:
sns.histplot(x=book['User Rating'], kde=False, bins=50)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16,10))
fig.delaxes(axs[1,1])

axs[0,0].hist(book['Reviews'], bins=50)
axs[0,1].hist(book['Price'], bins=50)
axs[1,0].hist(book['Year'], bins=50)
axs[0,0].title.set_text('Reviews')
axs[0,1].title.set_text('Price')
axs[1,0].title.set_text('Year')

plt.show()

In [ ]:
book.plot(x='Genre', y='Reviews', kind='bar')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,7))
ax[0].scatter('Price', 'User Rating', data=book, color='b')
ax[1].scatter('Reviews', 'User Rating', data=book, color='r')
plt.show()

In [ ]:
Fiction = book['Genre']=='Fiction'
Non_Fiction = book['Genre']=='Non Fiction'
plt.boxplot([book[Fiction]['User Rating'], book[Non_Fiction]['User Rating']])
plt.show()

In [ ]:
sns.jointplot(x=book['Price'], y=book['User Rating'], kind="kde")

***Creating new dataframe by copying the existing one so we can use it later without errors.***

In [ ]:
book_for_tree = book.copy(deep=True)
book_for_tree

***Generate Dummies***

In [ ]:
book=pd.get_dummies(book, drop_first=True, columns=['Year', 'Genre'])
book.head()

***Plotting Correlation***

In [ ]:
book.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))
sns.heatmap(book.corr(),annot=True,ax=ax)

In [ ]:
book.columns

# **Remove Outliers**# 

In [ ]:
plt.hist(book['Reviews'], bins=100)
outlier_limit = (book['Reviews'].mean() + 3*book['Reviews'].std())
plt.axvline(x=outlier_limit, color='r')
plt.show()

In [ ]:
book = book[book['Reviews'] <= outlier_limit]

# **Linear Regression**# 

In [ ]:
z = ['Year_2010',
       'Year_2011', 'Year_2012', 'Year_2013', 'Year_2014', 'Year_2015',
       'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019',
       'Genre_Non Fiction']
t = ['Reviews', 'Price', 'Year_2010']
y = book['User Rating'].values.reshape(-1,1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
for v in z:
    t.append(v)
    X = book[t]
    reg = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    y_pred_train = reg.predict(X_train)
    R_squared = reg.score(X_test, y_test)
    print(R_squared)

# **Decision Tree: To predict Genre**# 

In [ ]:
X1 = np.array(book_for_tree[['Reviews', 'Price', 'Year', 'User Rating']])
y1 = np.array(book_for_tree[['Genre']])
from sklearn import tree

tree = tree.DecisionTreeClassifier(max_depth=4)
tree = tree.fit(X1, y1)

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.30, random_state = 1) 

In [ ]:
tree.score(X1, y1)

In [ ]:
y1_pred = tree.predict(X1_test)
y1_pred

***More EDA***

In [ ]:
plt.figure(figsize=(16,4))
book.boxplot()
plt.show()

In [ ]:
book.describe()